<a href="https://colab.research.google.com/github/rashida048/ApacheSpark-BigDataAnalytics/blob/main/LinearRegression_task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 60 kB/s 
     |████████████████████████████████| 198 kB 52.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=731cbe522f9ffb07d17cab43b25cf9084ce77d88456a0b1d1645ec0c8804d505
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [ ]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [ ]:
rd = sc.textFile("taxi-data-sorted-small.csv.bz2")

In [ ]:
words = rd.map(lambda x: x.split(','))

In [ ]:
def isfloat(value):
    try:
        float(value)
        return True
    except:
        return False

In [ ]:
def correctRows(p):
    if(len(p)==17):
        if(isfloat(p[5]) and isfloat(p[11])):
            if(float(p[5]) !=0 and float(p[11]) !=0):
                return p

In [ ]:
wordsCorr = words.filter(lambda x: correctRows(x))

In [ ]:
dist_fair = wordsCorr.map(lambda x: (float(x[5]), float(x[11])))

In [ ]:
n = dist_fair.count()

In [ ]:
m = 0.1
b = 0.1
lr = 0.001

precision = 0.001
#n = 5000
const = -2/n


cost_list = []
m_list = [0.1]
b_list = [0.1]

for i in range(5):
  l = dist_fair.map(lambda x: (x[1] - (m*x[0] + b))**2).reduce(lambda x, y: x+y)
  #l = dist_fair.map(lambda x: (y_pred(x, m, b))**2).reduce(lambda x, y: x+y)
  #y_pred = dist_fair.map(lambda x: m*x[0] + b)
  del_m = dist_fair.map(lambda x: x[0] * (x[1] - (m*x[0] + b))).reduce(lambda x, y: x+y)
  m_cost = del_m*const
  del_b = dist_fair.map(lambda x: x[1] - (m*x[0] + b)).reduce(lambda x, y: x+y)
  b_cost = del_b*const
  m -= m_cost*lr
  #print(m)
  b -= b_cost*lr
  l = dist_fair.map(lambda x: (x[1] - (m*x[0] + b))**2).reduce(lambda x, y: x+y)
  print(l)
  m_list.append(m)
  b_list.append(b)

  if (abs(l - cost_list[-1]) <= precision):
    print("stopped at iteration", i)
    break
  cost_list.append(l)
total_list = (m_list, b_list, cost_list)

403452654.4420173
373247426.35937595
345488914.6984174
319978736.2523405
296534593.4419462
